## Setup

Add the project root and import the loss classes used in this notebook.


In [ ]:
import sys
sys.path.insert(0, "../../../src")

import numpy as np
from didgelab import (
    CompositeTairuaLoss,
    FrequencyTuningLoss,
    ScaleTuningLoss,
    PeakQuantityLoss,
    PeakAmplitudeLoss,
    QFactorLoss,
    ModalDensityLoss,
    IntegerHarmonicLoss,
    NearIntegerLoss,
    StretchedOddLoss,
    HighInharmonicLoss,
    HarmonicSplittingLoss,
    note_to_freq,
)


# Frequency Tuning Loss

**Purpose:** Align resonance peaks to specific target frequencies *and optionally* target impedances (e.g. drone and first overtone). Each target is matched to the nearest detected peak; the loss combines pitch error (in cents) and optionally amplitude error.

**Formula:**

$$L_{tune} = \sum_{i=1}^{N} w_i \cdot \left( \frac{|1200 \,\Delta \log_2 f_i|}{600} + [Z_{target,i} \neq -1] \cdot |Z_{target,i} - Z_{peak,i}| \right)$$

**Symbols:**
- $ w_i $: Weight for the $i$-th target.
- $ \Delta \log_2 f_i $: Log-frequency error (target vs. closest peak).
- 1200/600: Normalizes cent deviation (1 tritone = 1.0 base unit).
- $ Z_{target,i} $: Target normalized impedance (0–1). Use **−1 to ignore** impedance for that peak.
- $ Z_{peak,i} $: Actual normalized impedance at the matched peak.
- $ [Z \neq -1] $: Indicator; impedance term is zero when target is −1.

In [ ]:
# Target notes as MIDI (e.g. -31 = D1, -19 = D2) or frequencies in Hz
target_notes = [-31, -19]
target_freqs_hz = np.array([note_to_freq(n) for n in target_notes])
target_freqs_log = np.log2(target_freqs_hz)

# target_impedances: 0–1 per peak, or -1 to ignore amplitude for that peak
target_impedances = np.array([-1.0, -1.0])  # frequency-only; use e.g. [1.0, 0.7] to target amplitudes

freq_component = FrequencyTuningLoss(
    target_freqs_log=target_freqs_log,
    target_impedances=target_impedances,
    weights=[1.0, 1.0],  # same weight for drone and overtone
)
# Add to composite: loss.add_component("freq", freq_component)